# 1) Test split() function

In [2]:
import numpy as np
import matplotlib.pyplot as plt

show = False # Turn it to True if you want to plot input data in a image
# Input data --------------------------------------------
i_matrix = np.array([[0, 1, 0, 0], [0, 0, 1, 0]])
print('Original matrix: \n', i_matrix)
if show:
    plt.matshow(i_matrix)
    plt.title('Original matrix')
    plt.show()

# Function ----------------------------------------------
def split(matrix, depth):
    '''
    matrix: matrix shape(mxn)
    Depth: a number > 1
    '''
    n_tensor = [] # new created tensor
    for i in range(0, depth):
        # Creating the padding zero to compensate for the lost path of the shifted matrix
        r_matrix = len(matrix)
        pad = np.zeros((r_matrix, i))

        # 1: a new matrix is generated by concatenate the cut matrix and zero padding
        # 2: Collecting new matrices into a tensor
        n_tensor.append(np.concatenate((matrix[:, i:], pad), axis=1))
    return np.array(n_tensor)

# Show each lice of tensor after using split() function
print('\n' + '#'*15 + ' Print each slice of tensor after using split function ' + '#'*10)
tensor = split(i_matrix, 4)
for idx, i in enumerate(tensor):
    print(f'\nDepth number {idx+1}:')
    print(i)

Original matrix: 
 [[0 1 0 0]
 [0 0 1 0]]

############### Print each slice of tensor after using split function ##########

Depth number 1:
[[0. 1. 0. 0.]
 [0. 0. 1. 0.]]

Depth number 2:
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]]

Depth number 3:
[[0. 0. 0. 0.]
 [1. 0. 0. 0.]]

Depth number 4:
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]]


# 2) Test yield

In [5]:
import numpy as np
from scipy.io import wavfile
import matplotlib.pyplot as plt

def audio_source(name, window):
    sample_rate, samples = wavfile.read(name)
    begin = 0

    while begin < len(samples):
        end = min(begin + window, len(samples))
        yield samples[begin:end]
        begin = end

    print('len(samples) == end?', len(samples) == end)

# here len(piece) is just for showing each piece
generator = [len(piece) for piece in audio_source('wav/apple_and_lemmon.wav', 7000)]
print(list(generator))

len(samples) == end? True
[7000, 7000, 7000, 7000, 1376]


# 3) Test receiver() function

In [1]:
import numpy as np
from scipy.io import wavfile


def audio_source(name, window):
    sample_rate, samples = wavfile.read(name)
    begin = 0

    while begin < len(samples):
        end = min(begin + window, len(samples))
        yield samples[begin:end]
        begin = end

    print('len(samples) == end?', len(samples) == end)

def min_(data, s=False):
    yield np.min(data)

def max_(data, i=1):
    yield np.max(data)

def length_(data):
    yield len(data)

for data in audio_source('wav/apple_and_lemmon.wav', 7000):
    mi = next(min_(data))
    ma = next(max_(data))
    le = next(length_(data))
    print(f'\nMin: {mi}, max: {ma}, length {le}')


Min: -15045, max: 15733, length 7000

Min: -8768, max: 7947, length 7000

Min: -6330, max: 11097, length 7000

Min: -5881, max: 10405, length 7000

Min: -40, max: 1, length 1376
len(samples) == end? True


# 4) Test ma(), stair() and stra() while wav_to_wavelet() using yield

In [11]:
from utils.tools import stairway, wavelet_to_moving_average, matrix_to_vectors
from sklearn.neighbors import NearestCentroid
import numpy as np
from scipy.io import wavfile
from ssqueezepy import ssq_cwt
import numpy as np
import soundfile as sf

# Get wavelet from .wav file
def wav_to_wavelet(path, time_window=0.1):
    '''
    x: segment data (1D shape)
    time_window: time in seconds (s)
    '''
    track = sf.SoundFile(path)
    length = track.frames

    can_seek = track.seekable() # To enable access to a file
    if not can_seek:
        raise ValueError("Not compatible with seeking")

    begin = 0
    sr = track.samplerate # sample rate
    start_frame = sr * begin
    sampling_window = int(time_window*sr)
    
    while begin < length:
        track.seek(start_frame)
        audio_section = track.read(sampling_window)
        twx, wx, *_ = ssq_cwt(audio_section)  # use wx
        yield np.abs(wx)

        end = min(begin + sampling_window, length)
        begin = end


def stra(matrix, dis = 5):
    shape = matrix.shape
    x, y = matrix_to_vectors(matrix)
    x_list = np.array(np.arange(np.min(x), np.max(x), dis))
    y_list = np.array(np.arange(np.min(y), np.max(y), dis))
    x_1, y_1 = np.meshgrid(x_list, y_list)   # a grid is created by standard straighten

    x_train = np.array([x_1[:, i] for i in range(x_1.shape[1])]).reshape(-1, ).astype(np.int32)
    y_train = np.array([y_1[:, i] for i in range(y_1.shape[1])]).reshape(-1, )

    y_train = np.concatenate((y_train.reshape(-1, 1), x_train.reshape(-1, 1)), axis=-1)
    if len(np.unique(x_train)) < 2:
        return matrix
    ################################### Train with ML ###################################
    # This standard data ((y_train, x_train), x_train) is trained with a Machine Learning (ML) model(NearestCentroid). 
    # After that, using the machine learning
    # model predict original data ((y, x), y)
    clf = NearestCentroid()
    clf.fit(y_train, x_train)
    x = clf.predict(np.concatenate((y.reshape(-1, 1), x.reshape(-1, 1)), axis=-1))
    return x.reshape(shape)
    
path = 'wav/apple_and_lemmon.wav'
window = 1000 # pooling window in MA function
time_window=0.1
bins = 10 # number of bins in stairway function
dis = 200 # distance between columns in stra function

for idx, wx in enumerate(wav_to_wavelet(path, time_window)):
    ma_hist = stairway(wavelet_to_moving_average(wx, window), bins)
    print(f'Shape of stair segment {idx+1}: {ma_hist.shape}')
    ma_hist_stra = stra(ma_hist, dis = dis)
    print(f'Shape of straight segment {idx+1}: {ma_hist_stra.shape}\n')

Shape of stair segment 1: (245, 1401)
Shape of straight segment 1: (245, 1401)

Shape of stair segment 2: (245, 1401)
Shape of straight segment 2: (245, 1401)

Shape of stair segment 3: (245, 1401)
Shape of straight segment 3: (245, 1401)

Shape of stair segment 4: (245, 1401)
Shape of straight segment 4: (245, 1401)

Shape of stair segment 5: (245, 1401)
Shape of straight segment 5: (245, 1401)

Shape of stair segment 6: (245, 1401)
Shape of straight segment 6: (245, 1401)

Shape of stair segment 7: (245, 1401)
Shape of straight segment 7: (245, 1401)

Shape of stair segment 8: (245, 1401)
Shape of straight segment 8: (245, 1401)

Shape of stair segment 9: (245, 1401)
Shape of straight segment 9: (245, 1401)

Shape of stair segment 10: (245, 1401)
Shape of straight segment 10: (245, 1401)

Shape of stair segment 11: (245, 1401)
Shape of straight segment 11: (245, 1401)

Shape of stair segment 12: (245, 1401)
Shape of straight segment 12: (245, 1401)

Shape of stair segment 13: (245, 1